Dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

np.random.seed(42)

# Synthetic Construction Management Dataset
data = {
    'Project_ID': [f'P{i:03d}' for i in range(1, 21)],
    'Project_Cost_USD': np.random.randint(50_000, 500_000, 20).astype(float),
    'Duration_Days': np.random.randint(90, 600, 20),
    'Team_Size': np.random.randint(5, 40, 20),
    'Manager_Experience_Years': np.random.randint(1, 30, 20),
    'Project_Status': np.random.choice(['Completed', 'Ongoing', 'Delayed'], 20),
    'Client_Satisfaction': np.random.choice(['Excellent', 'Good', 'Poor'], 20)
}

df = pd.DataFrame(data)
df.loc[[0, 3], 'Project_Cost_USD'] = [2_000_000, 3_500_000]
df.loc[[1, 3], 'Client_Satisfaction'] = ['poor', 'Excelent']
df = pd.concat([df, df.loc[[9]]], ignore_index=True)


# Introduce Missing Data in 8 Different Ways
df.loc[1, 'Project_Cost_USD'] = np.nan       # 1. Listwise deletion example
df.loc[2, 'Duration_Days'] = np.nan          # 2. Pairwise deletion example
df.loc[3, 'Team_Size'] = np.nan              # 3. Mean/Median/Mode
df.loc[4, 'Manager_Experience_Years'] = np.nan # 4. Constant Value
df.loc[5, 'Client_Satisfaction'] = np.nan    # 5. Regression
df.loc[6, 'Project_Cost_USD'] = np.nan       # 6. KNN
df.loc[7, 'Duration_Days'] = np.nan          # 7. Multiple Imputation
df.loc[8, 'Project_Cost_USD'] = np.nan       # 8. Expert / Domain estimation

df.head()


1) Missing Data

In [ ]:
missing_summary = pd.DataFrame({
    'Missing_Count': df.isna().sum(),
})
print(missing_summary)

rows_with_nan = df.isna().any(axis=1).sum()
rows_with_nan_percent = df.isna().any(axis=1).mean()*100

print(f"Rows with at least one NaN: {rows_with_nan}, {rows_with_nan_percent}")

| # | Method              | When to Use                       | Pros                   | Cons                      |
| - | ------------------- | --------------------------------- | ---------------------- | ------------------------- |
| 1 | Listwise Deletion   | <5% missing, MCAR                 | Easy, clean            | Data loss                 |
| 2 | Pairwise Deletion   | Correlation-based analyses        | Keeps more data        | Complex interpretation    |
| 3 | Mean/Median/Mode    | MCAR or small gaps                | Simple, quick          | Distorts variance         |
| 4 | Constant Value      | Meaningful “Unknown” values       | Keeps structure        | Artificial bias           |
| 5 | Regression          | MAR data, linear relation         | Uses info efficiently  | Assumes linearity         |
| 6 | KNN                 | Nonlinear, small/moderate dataset | Preserves relations    | Slow on large data        |
| 7 | Multiple Imputation | MAR or mixed                      | Statistically rigorous | Computationally heavy     |
| 8 | Expert-Based        | MNAR or domain-driven             | Context-aware          | Subjective, not automated |


1-1) Listwise Deletion

In [ ]:
df_listwise = df.dropna()
print("\n1. Listwise Deletion:")
print(df_listwise)


1-2) Pairwise Deletion (no specific rows are permanently deleted, A row might be included/excluded in one analysis)

In [ ]:
numeric_df = df.select_dtypes(include=np.number)
corr_pairwise = numeric_df.corr(method='pearson', min_periods=1)
print("\n2. Pairwise Deletion (correlation using available pairs):")
print(corr_pairwise)

1-3) Mean/Median/Mode

In [ ]:
from scipy import stats
df_mean = df.copy()
df_mean['Team_Size'].hist()
print("skewness:", df_mean['Team_Size'].skew())
df_mean['Team_Size'].fillna(df_mean['Team_Size'].mean(), inplace=True)
print("\n3. Mean Imputation (Team_Size):")
print(df[['Project_ID', 'Team_Size']].head(6))
print(df_mean[['Project_ID', 'Team_Size']].head(6))


1-4) Constant Value Imputation

In [ ]:
df_constant = df.copy()
df_constant['Manager_Experience_Years'].fillna(0, inplace=True)
print("\n4. Constant Value Imputation (Manager_Experience_Years = 0):")
print(df[['Project_ID', 'Manager_Experience_Years']].head(6))
print(df_constant[['Project_ID', 'Manager_Experience_Years']].head(6))


1-5) Regression

In [ ]:
df_reg = df.copy()
train_data = df_reg.dropna(subset=['Project_Cost_USD', 'Manager_Experience_Years'])
reg = LinearRegression()
reg.fit(train_data[['Manager_Experience_Years']], train_data['Project_Cost_USD'])

missing_idx = df_reg['Project_Cost_USD'].isna()
predicted_costs = reg.predict(df_reg.loc[missing_idx, ['Manager_Experience_Years']].fillna(df_reg['Manager_Experience_Years'].mean()))
df_reg.loc[missing_idx, 'Project_Cost_USD'] = predicted_costs

print("\n5. Regression Imputation (predict Project_Cost_USD using Experience):")
print(df[['Project_ID', 'Project_Cost_USD']].head(8))
print(df_reg[['Project_ID', 'Project_Cost_USD']].head(8))


1-6) KNN

In [ ]:
df_knn = df.copy()
num_cols = ['Project_Cost_USD', 'Duration_Days', 'Team_Size', 'Manager_Experience_Years']
imputer = KNNImputer(n_neighbors=3)
df_knn[num_cols] = imputer.fit_transform(df_knn[num_cols])

print("\n6. KNN Imputation (nearest neighbor filling):")
print(df[num_cols].head(8))
print(df_knn[num_cols].head(8))


1-7) Multiple (Iterative) Imputation

In [ ]:
df_multi = df.copy()
iter_imputer = IterativeImputer(random_state=0)
df_multi[num_cols] = iter_imputer.fit_transform(df_multi[num_cols])

print("\n7. Multiple (Iterative) Imputation:")
print(df[num_cols].head(8))
print(df_multi[num_cols].head(8))


1-8) Expert-Based Estimation

In [ ]:
df_expert = df.copy()

# Estimate Project_Cost = Duration * Team_Size * 300
df_expert.loc[df_expert['Project_Cost_USD'].isna(), 'Project_Cost_USD'] = \
    df_expert['Duration_Days'] * df_expert['Team_Size'] * 300

print("\n8. Expert-Based Estimation (domain formula applied):")
print(df[['Project_ID', 'Project_Cost_USD']].head(10))
print(df_expert[['Project_ID', 'Project_Cost_USD']].head(10))


2) Inconsistencies

2-1) Duplicates

In [ ]:
duplicated_rows = df[df.duplicated(keep=False)]

print("✅ All Duplicated Rows (including originals):")
print(duplicated_rows)
print(f"\nTotal duplicated rows found: {duplicated_rows.shape[0]}")

df = df.drop_duplicates(keep='first', ignore_index=True)
df


2-2) Check for Data Type Inconsistencies

In [ ]:
for col in df.columns:
    if df[col].map(type).nunique() > 1:
        print(f"\n⚠️ Inconsistent types found in '{col}' column:")
        print(df[col].apply(type).value_counts())
        print(df[col].unique())

df['Client_Satisfaction'] = (
    df['Client_Satisfaction']
    .astype(str)
    .str.replace(r'[^0-9.]', '', regex=True)
    .replace('', np.nan)
    .astype(float)
)

print(df[col].apply(type).value_counts())
print(df[col].unique())

2-3) Outliers

In [ ]:
Q1 = df['Project_Cost_USD'].quantile(0.25)
Q3 = df['Project_Cost_USD'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_outliers = df[(df['Project_Cost_USD'] < lower_bound) | (df['Project_Cost_USD'] > upper_bound)]
print("Outliers (Before):\n", df_outliers)

df_removed = df[(df['Project_Cost_USD'] >= lower_bound) & (df['Project_Cost_USD'] <= upper_bound)]

# df_capped = df.copy()
# df_capped['Project_Cost_USD'] = np.where(df_capped['Project_Cost_USD'] > upper_bound, upper_bound,
#                                          np.where(df_capped['Project_Cost_USD'] < lower_bound, lower_bound,
#                                                   df_capped['Project_Cost_USD']))

# df_transformed = df.copy()
# df_transformed['Project_Cost_USD'] = np.log(df_transformed['Project_Cost_USD'])

# median_value = df['Project_Cost_USD'].median()
# df_imputed = df.copy()
# df_imputed.loc[(df['Project_Cost_USD'] < lower_bound) | (df['Project_Cost_USD'] > upper_bound), 'Project_Cost_USD'] = median_value

Q1 = df['Project_Cost_USD'].quantile(0.25)
Q3 = df['Project_Cost_USD'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_outliers = df_removed[(df_removed['Project_Cost_USD'] < lower_bound) | (df_removed['Project_Cost_USD'] > upper_bound)]
print("Outliers (After):\n", df_outliers)

3) Scaling

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# Select numerical columns
num_cols = ['Project_Cost_USD', 'Duration_Days', 'Team_Size', 'Manager_Experience_Years']

# Handle missing values with simple imputation (mean) for demonstration
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

# Function to visualize before and after scaling
def visualize_scaling(original, scaled, method_name):
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    sns.boxplot(data=original, ax=axes[0])
    axes[0].set_title('Before Scaling')
    sns.boxplot(data=scaled, ax=axes[1])
    axes[1].set_title(f'After {method_name} Scaling')
    plt.show()

    print(f"--- Statistics Before Scaling ---\n{original.describe()}\n")
    print(f"--- Statistics After {method_name} Scaling ---\n{pd.DataFrame(scaled, columns=original.columns).describe()}\n")

# 1️⃣ Min-Max Normalization
minmax_scaler = MinMaxScaler()
scaled_minmax = minmax_scaler.fit_transform(df[num_cols])
visualize_scaling(df[num_cols], scaled_minmax, 'Min-Max (Normalization)')

# 2️⃣ Standardization (Z-score)
standard_scaler = StandardScaler()
scaled_standard = standard_scaler.fit_transform(df[num_cols])
visualize_scaling(df[num_cols], scaled_standard, 'Standardization (Z-score)')

# 3️⃣ Robust Scaling (using median and IQR)
robust_scaler = RobustScaler()
scaled_robust = robust_scaler.fit_transform(df[num_cols])
visualize_scaling(df[num_cols], scaled_robust, 'Robust Scaling')